In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import seaborn as sns
import ccxt
import functions as fn
from data import fees_schedule, order_book

Origin:
- Exchange : Kraken
- Fee schedule : Maker fee : 10 bps - Taker fee : 20 bps
- Inventory : 100 BTC - 1M EUR

Destination:
- Exchange : Bitpanda
- Fee schedule : Maker fee : 15 bps - Taker fee : 30 bps
- Inventory : 100 BTC - 1M EUR

Strategy Parameters : 
- Replicate first 100 bps
- FIFO system at the exchange
- Take into account fees
- Take into account execution latency
- Process Queue
- Read Order Book

In [2]:
exchanges = ["kraken", "bitfinex"]
symbol = 'BTC/EUR'
expected_volume = 0

In [3]:
fees = [fees_schedule(exchange=i, symbol=symbol, expected_volume=expected_volume) for i in exchanges]
ob_data = pd.read_json('files/orderbooks_ray.json', orient='values', typ='series')

In [4]:
EO_ts_data = [i for i in ob_data[exchanges[0]].keys() if ob_data[exchanges[0]][i] != None]
ED_ts_data = [i for i in ob_data[exchanges[1]].keys() if ob_data[exchanges[1]][i] != None]

q1_results = fn.f_timestamps_info(ts_list_o=EO_ts_data, ts_list_d=ED_ts_data)

display('Timestamps in Origin data:')
display('First Timestamp: ' + q1_results["first_o"])
display('Last Timestamp: ' + q1_results["last_o"])
display('Total number of orderbooks: ' + str(q1_results["qty_o"]))

display('Timestamps in Destination data:')
display('First Timestamp: ' + q1_results["first_d"])
display('Last Timestamp: ' + q1_results["last_d"])
display('Total number of orderbooks: ' + str(q1_results["qty_d"]))

display('Exact match of Timestamps: ' + str(q1_results["exact_match"]["qty"]))
if q1_results["exact_match"]["qty"] == 0:
    display("no exact matches")
else:
    display('First 2 values are: ')
    display(q1_results["exact_match"]["values"][0].strftime("%Y-%m-%dT%H:%M:%S.%fZ"))
    display(q1_results["exact_match"]["values"][1].strftime("%Y-%m-%dT%H:%M:%S.%fZ"))
    display('Last 2 values are: ')
    display(q1_results["exact_match"]["values"][-1].strftime("%Y-%m-%dT%H:%M:%S.%fZ"))
    display(q1_results["exact_match"]["values"][-2].strftime("%Y-%m-%dT%H:%M:%S.%fZ"))



'Timestamps in Origin data:'

'First Timestamp: 2021-11-24T15:15:52.328000Z'

'Last Timestamp: 2021-11-24T15:17:46.627000Z'

'Total number of orderbooks: 39'

'Timestamps in Destination data:'

...

'2021-11-24T15:16:28.424000Z'

'2021-11-24T15:16:31.432000Z'

'Last 2 values are: '

'2021-11-24T15:16:31.432000Z'

'2021-11-24T15:16:28.424000Z'

In [5]:
fees[1]

{'taker': 0.002, 'maker': 0.001}

In [6]:
ex_O = pd.DataFrame(ob_data[exchanges[0]]['2021-11-24T15:16:31.432Z'])
ex_D = pd.DataFrame(ob_data[exchanges[1]]['2021-11-24T15:16:31.432Z'])

In [7]:
display('Highest Bid Prices')
display('Destination: '+ exchanges[1]+ ' ' + str(max(ex_D.bid)))
display('Origin: ' + exchanges[0]+ ' ' + str(max(ex_O.bid)))

'Highest Bid Prices'

'Destination: bitfinex 56268.0'

'Origin: kraken 56276.0'

In [8]:
display('Lowest Ask Prices')
display('Destination: ' + exchanges[1]+ ' ' + str(min(ex_D.ask)))
display('Origin: ' + exchanges[0]+ ' ' +str(min(ex_O.ask)))

'Lowest Ask Prices'

'Destination: bitfinex 56273.0'

'Origin: kraken 56276.1'

Puedo vender en origin 56,276 y comprar en destination a 56,273 teniendo un profit de 3EUR

Puedo comprar en origin en 56,276.1 y vender en destination en 56,268 teniendo un profit de -8.1EUR

$ maker_{bid} = taker_{bid} * (1 - taker_{fee}) / (1 + maker_{fee}) $

$ taker_{bid} * (1 + taker_{fee}) = maker_{ask} * (1 - maker_{fee})$

In [9]:
fees[0]

{'taker': 0.0026, 'maker': 0.0016}

In [10]:
fees[1]

{'taker': 0.002, 'maker': 0.001}

In [12]:
origin_bid = 56276.0
taker_fee = 0.0026
maker_fee = 0.001

In [14]:
maker_bid = origin_bid*(1-taker_fee)/(1+maker_fee)
maker_bid

56073.6087912088

In [15]:
ex_D

,ask_size,ask,bid,bid_size,spread
0,0.035600,56273.0,56268.0,0.000070,5.0
1,0.187329,56275.0,56267.0,0.420000,8.0
2,0.182073,56281.0,56264.0,0.001985,17.0
3,0.074227,56282.0,56263.0,0.002481,19.0
4,0.059343,56286.0,56262.0,0.009923,24.0
5,0.190300,56287.0,56261.0,0.168181,26.0
6,0.010000,56289.0,56259.0,0.049619,30.0
7,0.948159,56290.0,56257.0,0.595527,33.0
8,0.100000,56291.0,56256.0,0.009100,35.0
9,0.009792,56292.0,56255.0,0.000330,37.0
